# IGDB Embeddings pgvector Demo - Inference & Search

In [1]:
!pip install "psycopg[binary]" --quiet

In [2]:
import sagemaker
import boto3
import json
import pandas as pd
import psycopg
from sagemaker.huggingface.model import HuggingFacePredictor
from IPython.display import display, HTML

In [3]:
# Create SageMaker Session
sess = sagemaker.Session()

# Create SecretsManager client
secretsmanager = boto3.client('secretsmanager')

## Retrive all variables from Notebook Instance's tags

In [4]:
# Get this Notebook Instance's ARN
NOTEBOOK_ARN = !jq '.ResourceArn' /opt/ml/metadata/resource-metadata.json --raw-output
NOTEBOOK_ARN = NOTEBOOK_ARN[0]

# Get the tags of this Notebook Instance
tags = sess.sagemaker_client.list_tags(ResourceArn=NOTEBOOK_ARN)['Tags']

# Filter out the keys that contains the necessary information
DB_SECRET_ARN = list(filter(lambda x: x['Key'] == 'VAR_DB_SECRET_ARN', tags))[0]['Value']
MODEL_ENDPOINT = list(filter(lambda x: x['Key'] == 'VAR_MODEL_ENDPOINT', tags))[0]['Value']

## Inference with Model Endpoint

In [5]:
# Get database credentials
secretsmanager = boto3.client('secretsmanager')

db_secret = secretsmanager.get_secret_value(
    SecretId=DB_SECRET_ARN
)
db_secret_string = json.loads(db_secret['SecretString'])
db_secret_string
db_host = db_secret_string['host']
db_port = db_secret_string['port']
db_user = db_secret_string['username']
db_pass = db_secret_string['password']

In [6]:
# Get the existing model
predictor = HuggingFacePredictor(MODEL_ENDPOINT, sagemaker_session=sess)

In [7]:
# Set question
QUESTION = "Zelda Tears of the Kingdom"

# Transform the question to embedding
data = {"inputs": QUESTION}
prediction = predictor.predict(data=data)
vector = prediction['vectors']

# Make a Query
with psycopg.connect(host=db_host, user=db_user, password=db_pass, port=db_port, connect_timeout=10, autocommit=True) as conn:
    with conn.cursor() as cur:
        # # Official answer but FAILED
        # cur.execute("""SELECT igdb_id, name, summary, description, url, artwork_hash, screenshot_hash, description_embeddings
        #                 FROM igdb 
        #                ORDER BY descriptions_embeddings <-> %s::vector limit 5;""", 
        #             (np.array(vector),))

        # Ref: https://github.com/pgvector/pgvector-python/issues/4#issuecomment-1474930998
        cur.execute("""SELECT igdb_id, name, summary, description, url, artwork_hash, screenshot_hash
                         FROM igdb 
                        ORDER BY description_embeddings <-> %s::vector limit 5;""", 
                    (vector,))

        responses = cur.fetchall()

# Display the result as HTML
def formatter_image(hash):
    return f'<img src="https://images.igdb.com/igdb/image/upload/t_screenshot_med/{hash}.jpg">'

def formatter_url(url):
    return f'<a href={url} target="_blank" rel="noopener noreferrer">Link to IGDB</a>'

result = pd.DataFrame(responses)
result.columns = ("IGDB ID", "Name", "Summary", "Description", "IGDB Page", "Artwork", "Screenshot",)
display(HTML(result.to_html(escape=False, formatters={
    "IGDB Page": formatter_url,
    "Artwork": formatter_image,
    "Screenshot": formatter_image,
})))

,IGDB ID,Name,Summary,Description,IGDB Page,Artwork,Screenshot
0,237289,The Legend of Zelda: Tears of the Kingdom - Collector's Edition,Includes:\n- The Legend of Zelda: Tears of the Kingdom (physical version)\n- Artbook\n- SteelBook case\n- Steel Poster\n- A set of four pin badges.,"Title: ""The Legend of Zelda: Tears of the Kingdom - Collector's Edition"" Summary: Includes:\n- The Legend of Zelda: Tears of the Kingdom (physical version)\n- Artbook\n- SteelBook case\n- Steel Poster\n- A set of four pin badges. Released on May 12, 2023. Publishers: Nintendo. Platforms: Nintendo Switch. Game Modes: Single player. Genres: Role-playing (RPG), Adventure. Themes: Action, Fantasy, Open world. Player Perspectives: Third person.",Link to IGDB,,
1,119388,The Legend of Zelda: Tears of the Kingdom,The Legend of Zelda: Tears of the Kingdom is the sequel to The Legend of Zelda: Breath of the Wild. The setting for Link’s adventure has been expanded to include the skies above the vast lands of Hyrule.,"Title: ""The Legend of Zelda: Tears of the Kingdom"" Summary: The Legend of Zelda: Tears of the Kingdom is the sequel to The Legend of Zelda: Breath of the Wild. The setting for Link’s adventure has been expanded to include the skies above the vast lands of Hyrule. Released on May 12, 2023. Publishers: Nintendo. Platforms: Nintendo Switch. Game Modes: Single player. Genres: Adventure. Themes: Action, Fantasy, Science fiction, Open world. Player Perspectives: Third person.",Link to IGDB,,
2,252906,The Legend of Zelda: Tears of the Kingdom - Better Sages Mod,"A mod which changes the appearance and behavior of the Sages companions in the game, making them stronger and faster, removing their cooldown and making their appearance the same as their original Secret Stone holders.","Title: ""The Legend of Zelda: Tears of the Kingdom - Better Sages Mod"" Summary: A mod which changes the appearance and behavior of the Sages companions in the game, making them stronger and faster, removing their cooldown and making their appearance the same as their original Secret Stone holders. Released on May 01, 2023. Publishers: ShrineFox. Platforms: Nintendo Switch. Game Modes: Single player. Genres: Adventure. Themes: Action, Fantasy, Science fiction. Player Perspectives: Third person.",Link to IGDB,,
3,223793,Doraemon Story of Seasons: Friends of the Great Kingdom - FGK DLC 1,Includes:\n- 5 outfits for Noby and Friends\n- 8 types of furniture\n- 1 mini-storyllllllllllllllllllllllllllll,"Title: ""Doraemon Story of Seasons: Friends of the Great Kingdom - FGK DLC 1"" Summary: Includes:\n- 5 outfits for Noby and Friends\n- 8 types of furniture\n- 1 mini-storyllllllllllllllllllllllllllll Released on Nov 02, 2022. Publishers: unknown. Platforms: Nintendo Switch. Game Modes: unknown. Genres: unknown. Themes: unknown. Player Perspectives: unknown.",Link to IGDB,,
4,209344,Sakura Fantasy,Become a cute fantasy knight in a magical kingdom of beautiful women,"Title: ""Sakura Fantasy"" Summary: Become a cute fantasy knight in a magical kingdom of beautiful women Released on Jul 22, 2022. Publishers: unknown. Platforms: Nintendo Switch. Game Modes: Single player. Genres: Adventure. Themes: unknown. Player Perspectives: unknown.",Link to IGDB,,
